In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms_jakub import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
dataset = GermanDataset(device=device, sensitive_feature_labels=["Age", "Sex"])

print(dataset.sensitive_attrs)
#print(dataset.XZ_train[:5])

[[0, 1], [0, 1]]


In [4]:
def generate_res():
    test = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    return train, val, test

def append_res(l,acc,ei,be,dp,eo,eodd):
    l['accuracy'].append(acc)
    l['ei_disparity'].append(ei)
    l['be_disparity'].append(be)
    l['dp_disparity'].append(dp)
    l['eo_disparity'].append(eo)
    l['eodd_disparity'].append(eodd)

def get_res(l):
    res = {}
    res['accuracy_mean'] = np.mean(l['accuracy'])
    res['accuracy_var'] = np.std(l['accuracy'])
    res['accuracy_list'] = l['accuracy']
    res['ei_mean'] = np.mean(l['ei_disparity'])
    res['ei_var'] = np.std(l['ei_disparity'])
    res['ei_list'] = l['ei_disparity']
    res['be_mean'] = np.mean(l['be_disparity'])
    res['be_var'] = np.std(l['be_disparity'])
    res['be_list'] = l['be_disparity']
    res['dp_mean'] = np.mean(l['dp_disparity'])
    res['dp_var'] = np.std(l['dp_disparity'])
    res['dp_list'] = l['dp_disparity']
    res['eo_mean'] = np.mean(l['eo_disparity'])
    res['eo_var'] = np.std(l['eo_disparity'])
    res['eo_list'] = l['eo_disparity']
    res['eodd_mean'] = np.mean(l['eodd_disparity'])
    res['eodd_var'] = np.std(l['eodd_disparity'])
    res['eodd_list'] = l['eodd_disparity']
    return res

def lr_kde_model_runner(dataset, hp, seeds):
    
    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):

    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    
    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, BE_hp_fb, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI_fc = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    _, _, EI_kde = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    _, _, BE_fb = lr_fb_model_runner(dataset, BE_hp_fb, seeds)
    _, _, EI_fb = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD, EI_fc, EI_kde, EI_fb, BE_fb

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 0.5

kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


c:\Users\User\s1\fact\ei_fairness_reproducibility\notebooks\..\utils\dataloaders.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  return torch.FloatTensor(X).to(device), torch.FloatTensor(Y).to(device), torch.FloatTensor(Z).to(device), torch.FloatTensor(XZ).to(device)
Training: 100%|██████████| 100/100 [00:03<00:00, 25.24epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 39.80epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 60.22epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 42.91epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          be         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  ---------  ---------
         0.0001          0          0.317188         0.31875  0.0105909  0.00902778  0.05       0.115044   0.115044
         0.001           0          0.673438         0.6875   0.127273   0.00347222  0.0423611  0.0980531  0.0980531
         0.01            0          0.75             0.78125  0.125      0.0673611   0.183333   0.106903   0.167021
         0.1             0          0.779687         0.75625  0.180392   0.034375    0.209722   0.124956   0.203191


The learning rate is decided as 0.005.


In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0

#### BE Hyperparameter Test

In [7]:
BE_hp_test = SGD_hp_test.copy()
BE_hp_test['learning_rate'] = [0.1]
BE_hp_test['lambda_'] = [0, 0.1, 0.2, 0.3, 0.4]
BE_hp_test['fairness'] = 'BE'

fb_hyperparameter_test(dataset, BE_hp_test, seed=0)

training seed 0 started


Training:   0%|          | 0/100 [00:00<?, ?epochs/s]

Training:   0%|          | 0/100 [43:30<?, ?epochs/s]


KeyboardInterrupt: 

In [ ]:
BE_hp_fb = BE_hp_test.copy()
BE_hp_fb['learning_rate'] = 0.1
BE_hp_fb['lambda_'] = 0.2


#### EI Hyperparameter Test

In [ ]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 37.14epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 38.71epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 47.32epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:04<00:00, 24.97epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         be        dp         eo      eodd
---------------  ---------  ----------------  --------------  ---------  ---------  --------  ---------  --------
            0.1       0             0.779687         0.75625  0.180392   0.034375   0.209722  0.124956   0.203191
            0.1       0.2           0.773438         0.74375  0.165865   0.034375   0.177778  0.124956   0.145745
            0.1       0.6           0.765625         0.7375   0.0960591  0.0104167  0.129861  0.0938053  0.138298
            0.1       0.8           0.746875         0.74375  0.0409091  0.021875   0.213889  0.151858   0.168085
            0.1       0.9           0.460938         0.50625  0.0723684  0.05625    0.213889  0.282124   0.282124
            0.1       0.95          0.7              0.70625  0          0          0         0          0


The lambda for EI is decided as 0.9.

In [ ]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.1
EI_hp_kde['lambda_'] = 0.95

In [ ]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.99]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:02<00:00, 41.69epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:04<00:00, 21.07epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:03<00:00, 29.62epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:04<00:00, 21.60epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         be        dp        eo      eodd
---------------  ---------  ----------------  --------------  ---------  ---------  --------  --------  --------
            0.1       0             0.779687         0.75625  0.180392   0.034375   0.209722  0.124956  0.203191
            0.1       0.6           0.775            0.75625  0.143137   0.040625   0.209722  0.124956  0.203191
            0.1       0.8           0.773438         0.75     0.19697    0.046875   0.19375   0.124956  0.174468
            0.1       0.9           0.770312         0.7375   0.268398   0.0625     0.19375   0.156106  0.156106
            0.1       0.95          0.771875         0.73125  0.257212   0.053125   0.177778  0.156106  0.156106
            0.1       0.99          0.76875          0.725    0.0295567  0.0131944  0.129861  0.124956  0.159574


In [ ]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.1
EI_hp_fc['lambda_'] = 0.99

In [ ]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.1, 0.2, 0.4, 0.6]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   3%|▎         | 3/100 [00:00<00:03, 25.52epochs/s]

Training: 100%|██████████| 100/100 [00:02<00:00, 48.79epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 53.08epochs/s]


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:03<00:00, 30.40epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 54.04epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         be        dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  --------  ---------  ---------
            0.1        0            0.779687         0.75625  0.180392   0.034375   0.209722  0.124956   0.203191
            0.1        0.1          0.775            0.74375  0.183333   0.0361111  0.168056  0.133805   0.133805
            0.1        0.2          0.765625         0.75625  0.0882353  0.0458333  0.143056  0.0849558  0.103191
            0.1        0.4          0.746875         0.74375  0.166667   0.0583333  0.109722  0.106903   0.144681
            0.1        0.6          0.707812         0.70625  0.0555556  0.01875    0.0125    0.025592   0.0414894


In [ ]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.1
EI_hp_fb['lambda_'] = 0.1

#### Model training

In [ ]:
seeds = np.arange(1,6)

SGD, EI_fc, EI_kde, EI_fb, BE_fb = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, BE_hp_fb ,seeds)

C:\Users\User\AppData\Local\Temp\ipykernel_12108\392662720.py:60: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:02<00:00, 40.79epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:04<00:00, 24.56epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:03<00:00, 33.09epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:04<00:00, 24.72epochs/s]

Training finished for all seeds.


In [ ]:
result = []
models = ["SGD", "EI FC", "EI KDE", "EI FB", "BE FB"]
sol = [SGD, EI_fc, EI_kde, EI_fb, BE_fb]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['be_mean'])
    c.append(res['be_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var","be_mean","be_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    be_mean     be_var    dp_mean      dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ----------  ---------  ---------  ---------  ----------  ---------  ----------  -----------  ----------
SGD                0.776      0.00583095  0.033636   0.00455034  0.105818   0.0190896  0.107227   0.0190269   0.109463   0.0151374     0.128242    0.0267759
EI FC              0.786      0.0152971   0.0633268  0.0281924   0.0747609  0.0141063  0.0682884  0.008949    0.0799483  0.0234947     0.083551    0.0166657
EI KDE             0.722      0.0218174   0.0620284  0.0231114   0.0739394  0.0290602  0.0721358  0.0310583   0.0912719  0.0380076     0.0936493   0.0344409
EI FB              0.784      0.0115758   0.0291196  0.0138806   0.0642884  0.0082529  0.0631504  0.0107851   0.0799483  0.0109684     0.0799483   0.0109684
BE FB              0.786      0.00969536  0.0415201  0.007